In [ ]:
!pip install emoji
!pip install stop-words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=b7f2b986f1276c42c5d61f4438b92b244601caf1853771286b756b9403ea6784
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,EarlyStoppingCallback
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import re
import emoji
import string
from nltk.corpus import stopwords
from stop_words import get_stop_words
import nltk

nltk.download('punkt')
nltk.download('stopwords')
tqdm.pandas()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('punkt', download_dir='/root/nltk_data')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dfb = pd.read_csv('SarcasDetection.csv')
dfb = dfb[['Comments', 'Label']]
dfb.columns = ['text','label']
dfb = dfb.drop_duplicates(subset='text').reset_index(drop=True)
dfb

,text,label
0,আমি মনে করি যখন মেয়েরা উদ্দেশ্য অনুসারে বোবা ...,1.0
1,আমি যখন কোনও গুরুত্বপূর্ণ প্রশ্ন জিজ্ঞাসা করি ...,1.0
2,বাহ ... আমি দেখতে দেখতে পাচ্ছি সত্যিই দোষী দোষ...,1.0
3,"হাহ, মজার বিষয়।#আনোয়েড #গ্রুআপ",1.0
4,থান্ডার আমাকে 830 এ জেগে উঠলে এটি ভালবাসুন,1.0
...,...,...
4633,আলগা পিরিতি। কিন্তু নিজের জেলার মানুষ কষ্ট করত...,0.0
4634,ফেইসবুক ছাড়া আপনাদের আর কোথাও দেখা মেলেনা।,0.0
4635,শুধু ঢাকাই বসে বসে বকবক করা ছাড়া কিছু করবে না ...,0.0
4636,সংবাদ পড়া বাদ দিয়ে মানুষের পাশে দাঁড়ান।,0.0


In [ ]:
dfhs = pd.read_csv('Sarcasm_Hindi_Tweets-SARCASTIC.csv')
dfhns = pd.read_csv('Sarcasm_Hindi_Tweets-NON-SARCASTIC.csv')
dfhs = dfhs[['text']]
dfhns = dfhns[['text']]
dfhs['label'] = 1.0
dfhns['label'] = 0.0
dfh = pd.concat([dfhs, dfhns], axis=0)
dfh = dfh.drop_duplicates(subset='text').reset_index(drop=True)
dfh

,text,label
0,"हमारे गांव में एक मास्टर जी रहते थे, वो मोहल्ल...",1.0
1,Dil hi saaf rakhlo... \nShakal ka to vaise bhi...,1.0
2,घोड़ो को ना मिली घास और गधे खा रहे चवनप्रास😃😃\...,1.0
3,पिछली सरकार के 10 साल के कार्यकाल में 171 जवा...,1.0
4,"""Akele aaye the akele hi jayengy"" ye kahawat a...",1.0
...,...,...
1127,https://t.co/HnIURjsvMT अंतर्राष्ट्रीय योग दिव...,0.0
1128,@Jaihindsonu59 @SinghChunni @SATYAPR27477125 क...,0.0
1129,https://t.co/qzgPRYlS1h प्रदेश में आज से कोरोन...,0.0
1130,"Zyada achey b mat bano, yad rkho k meethey pha...",0.0


In [ ]:
dfhsr = pd.read_csv('S Reddit Hindi.csv')
dfhnsr = pd.read_csv('NS Reddit Hindi.csv')
dfhsr = dfhsr[['comment']]
dfhnsr = dfhnsr[['comment']]
dfhsr.columns = ['text']
dfhnsr.columns = ['text']
dfhsr['label'] = 1.0
dfhnsr['label'] = 0.0
dfhr = pd.concat([dfhsr, dfhnsr], axis=0)
dfhr = dfhr.drop_duplicates(subset='text').reset_index(drop=True)
dfhr

<ipython-input-14-8236e455a7c4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfhsr['label'] = 1.0


,text,label
0,Not अनैटमी ?,1.0
1,>I’m kannad but I’d find it extremely difficul...,1.0
2,not quite there but maybe something like this...,1.0
3,"हिंदी एक जीवित भाषा है जिसने संस्कृत, फ़ारसी, ...",1.0
4,Ever heard of synonyms? \n\nAnd no सुखी is NOT...,1.0
...,...,...
350,बॉलिवुड़ फिल्में देखो। चाहते हो तो मुझे रेडिट ...,0.0
351,इनमें से एक भी अपनी बहन को भेजा तो वो पूछेगी क...,0.0
352,"“नही, पर अब जब बात आ ही गई है तो क्या मुझे थोड...",0.0
353,इन **अन्तरजाल स्रोतों** का वाचन किया जा सकता ह...,0.0


In [ ]:
dfbsr = pd.read_csv('S Reddit Bangla.csv')
dfbnsr = pd.read_csv('NS Reddit Bangla.csv')
dfbsr = dfbsr[['comment']]
dfbnsr = dfbnsr[['comment']]
dfbsr.columns = ['text']
dfbnsr.columns = ['text']
dfbsr['label'] = 1.0
dfbnsr['label'] = 0.0
dfbr = pd.concat([dfbsr, dfbnsr], axis=0)
dfbr = dfbr.drop_duplicates(subset='text').reset_index(drop=True)
dfbr

,text,label
0,আমরা ক্ষমতায়? হাসিনা গুলির চিন্তা না করলে আরো ...,1.0
1,আমারে টোকাই বললে রুল ভঙ্গ না হইলে আপনাকে শিবির...,1.0
2,এটা খুবই স্বাভাবিক। রুমমেট নির্বাচনে তো অবশ্যই...,1.0
3,৪.৩. পুলিশি জুলুম (১০ টি ভিডিও)\n\nপুরো আন্দোল...,1.0
4,> ইসলামিক সংবিধান এ যদি সমগ্র রাষ্ট্র পরিচালনা...,1.0
...,...,...
2085,"এমা, ভগবান পাপ দিলে??!",0.0
2086,"না না, আমি কিছু বলছি না তো, শুধু ভাবছি, বেচারি...",0.0
2087,🥺 মা এইটুকুনি দুষ্টুমির জন্য কিছু করবে না। ভয়...,0.0
2088,যদি একটু কান মলে দেন??!\n\n\n\n🤭🤭🤭,0.0


In [ ]:
bangla_stopwords=["অতএব","অথচ","অথবা","অনুযায়ী","অনেক","অনেকে","অনেকেই","অন্তত","অন্য","অবধি","অবশ্য","অর্থাত","আই","আগামী","আগে",
                  "আগেই","আছে","আজ","আদ্যভাগে","আপনার","আপনি","আবার","আমরা","আমাকে","আমাদের","আমার","আমি","আর","আরও",
                  "ই","ইত্যাদি","ইহা","উচিত","উত্তর","উনি","উপর","উপরে","এ","এঁদের","এঁরা","এই","একই","একটি","একবার","একে","এক্","এখন",
                  "এখনও","এখানে","এখানেই","এটা","এটাই","এটি","এত","এতটাই","এতে","এদের","এব","এবং","এবার","এমন","এমনকী","এমনি","এর",
                  "এরা","এল","এস","এসে","ঐ","ও","ওঁদের","ওঁর","ওঁরা","ওই","ওকে","ওখানে","ওদের","ওর","ওরা","কখনও","কত","কবে","কমনে",
                  "কয়েক","কয়েকটি","করছে","করছেন","করতে","করবে","করবেন","করলে","করলেন","করা","করাই","করায়","করার","করি","করিতে","করিয়া",
                  "করিয়ে","করে","করেই","করেছিলেন","করেছে","করেছেন","করেন","কাউকে","কাছ","কাছে","কাজ","কাজে","কারও","কারণ","কি","কিংবা","কিছু",
                  "কিছুই","কিন্তু","কী","কে","কেউ","কেউই","কেখা","কেন","কোটি","কোন","কোনও","কোনো","ক্ষেত্রে","কয়েক","খুব","গিয়ে","গিয়েছে","গিয়ে",
                  "গুলি","গেছে","গেল","গেলে","গোটা","চলে","চান","চায়","চার","চালু","চেয়ে","চেষ্টা","ছাড়া","ছাড়াও","ছিল","ছিলেন","জন","জনকে","জনের",
                  "জন্য","জন্যওজে","জানতে","জানা","জানানো","জানায়","জানিয়ে","জানিয়েছে","জে","জ্নজন","টি","ঠিক","তখন","তত","তথা","তবু","তবে",
                  "তা","তাঁকে","তাঁদের","তাঁর","তাঁরা","তাঁাহারা","তাই","তাও","তাকে","তাতে","তাদের","তার","তারপর","তারা","তারৈ","তাহলে","তাহা","তাহাতে",
                  "তাহার","তিনঐ","তিনি","তিনিও","তুমি","তুলে","তেমন","তো","তোমার","থাকবে","থাকবেন","থাকা","থাকায়","থাকে","থাকেন","থেকে","থেকেই",
                  "থেকেও","দিকে","দিতে","দিন","দিয়ে","দিয়েছে","দিয়েছেন","দিলেন","দু","দুই","দুটি","দুটো","দেওয়া","দেওয়ার","দেওয়া","দেখতে","দেখা","দেখে",
                  "দেন","দেয়","দ্বারা","ধরা","ধরে","ধামার","নতুন","নয়","না","নাই","নাকি","নাগাদ","নানা","নিজে","নিজেই","নিজেদের","নিজের","নিতে","নিয়ে",
                  "নিয়ে","নেই","নেওয়া","নেওয়ার","নেওয়া","নয়","পক্ষে","পর","পরে","পরেই","পরেও","পর্যন্ত","পাওয়া","পাচ","পারি","পারে","পারেন","পি","পেয়ে",
                  "পেয়্র্","প্রতি","প্রথম","প্রভৃতি","প্রযন্ত","প্রাথমিক","প্রায়","প্রায়","ফলে","ফিরে","ফের","বক্তব্য","বদলে","বন","বরং","বলতে","বলল","বললেন","বলা",
                  "বলে","বলেছেন","বলেন","বসে","বহু","বা","বাদে","বার","বি","বিনা","বিভিন্ন","বিশেষ","বিষয়টি","বেশ","বেশি","ব্যবহার","ব্যাপারে","ভাবে","ভাবেই",
                  "মতো","মতোই","মধ্যভাগে","মধ্যে","মধ্যেই","মধ্যেও","মনে","মাত্র","মাধ্যমে","মোট","মোটেই","যখন","যত","যতটা","যথেষ্ট","যদি","যদিও","যা","যাঁর",
                  "যাঁরা","যাওয়া","যাওয়ার","যাওয়া","যাকে","যাচ্ছে","যাতে","যাদের","যান","যাবে","যায়","যার","যারা","যিনি","যে","যেখানে","যেতে","যেন","যেমন","র",
                  "রকম","রয়েছে","রাখা","রেখে","লক্ষ","শুধু","শুরু","সঙ্গে","সঙ্গেও","সব","সবার","সমস্ত","সম্প্রতি","সহ","সহিত","সাধারণ","সামনে","সি","সুতরাং","সে",
                  "সেই","সেখান","সেখানে","সেটা","সেটাই","সেটাও","সেটি","স্পষ্ট","স্বয়ং","হইতে","হইবে","হইয়া","হওয়া","হওয়ায়","হওয়ার","হচ্ছে","হত","হতে","হতেই",
                  "হন","হবে","হবেন","হয়","হয়তো","হয়নি","হয়ে","হয়েই","হয়েছিল","হয়েছে","হয়েছেন","হল","হলে","হলেই","হলেও","হলো","হাজার","হিসাবে","হৈলে",
                  "হোক","হয়"]

hindi_stop_words = get_stop_words('hi')
english_stopwords= stopwords.words('english')

stopwords_set = set(english_stopwords + hindi_stop_words + bangla_stopwords)

In [ ]:
url_pattern = re.compile(r'((www\.[\S]+)|(https?://[\S]+))')
mention_pattern = re.compile(r'@\w+')
space_pattern = re.compile(r'\s+')
hashtag_pattern = re.compile(r'#(\w+)')

# def process_text(text):
#     text = text.lower()
#     text = url_pattern.sub('', text)
#     text = mention_pattern.sub('', text)
#     text = space_pattern.sub(' ', text)
#     text = hashtag_pattern.sub(r'\1', text)
#     text = emoji.demojize(text)
#     #tokenized_text = nltk.word_tokenize(text) # check if tokenisation here is needed as bert tokeniser is used later
#     return ' '.join([word for word in text if word not in string.punctuation and word not in stopwords_set])


def process_text(text):
    text = text.lower()
    text = url_pattern.sub('', text)
    text = mention_pattern.sub('', text)
    text = space_pattern.sub(' ', text)
    text = hashtag_pattern.sub(r'\1', text)
    text = emoji.demojize(text)
    words = text.split()  # Split text into words
    return ' '.join([word for word in words if word not in string.punctuation and word not in stopwords_set])

In [ ]:
tqdm.pandas()
dfb['text'] = dfb['text'].progress_apply(process_text)
dfh['text'] = dfh['text'].progress_apply(process_text)
dfbr['text'] = dfbr['text'].progress_apply(process_text)
dfhr['text'] = dfhr['text'].progress_apply(process_text)

  0%|          | 0/4638 [00:00<?, ?it/s]

  0%|          | 0/1132 [00:00<?, ?it/s]

  0%|          | 0/2090 [00:00<?, ?it/s]

  0%|          | 0/355 [00:00<?, ?it/s]

In [ ]:
df = pd.concat([dfb, dfh, dfbr, dfhr], axis=0)
output_file_path = 'final_full_data_main.csv'
df.to_csv(output_file_path, index=False)
df

,text,label
0,মেয়েরা উদ্দেশ্য অনুসারে বোবা বোবা আচরণ সুন্দর,1.0
1,গুরুত্বপূর্ণ প্রশ্ন জিজ্ঞাসা সত্যিই পছন্দ করি।...,1.0
2,বাহ ... পাচ্ছি সত্যিই দোষী দোষী সাব্যস্তনোটপ্রেসড,1.0
3,"হাহ, মজার বিষয়।আনোয়েড গ্রুআপ",1.0
4,থান্ডার 830 জেগে উঠলে ভালবাসুন,1.0
...,...,...
350,बॉलिवुड़ फिल्में देखो। चाहते मुझे रेडिट मैंसेज...,0.0
351,इनमें बहन भेजा वो पूछेगी कहीं मेरी तबियत खराब,0.0
352,"“नही, अब बात आ गई क्या मुझे थोड़े रूपये मिल है?""",0.0
353,**अन्तरजाल स्रोतों** वाचन हिन्दी साहित्य हर दौ...,0.0


In [ ]:
label_counts = df['label'].value_counts()
print("Count of tweets with label 1.0 (sarcastic):", label_counts[1.0])
print("Count of tweets with label 0.0 (non-sarcastic):", label_counts[0.0])

Count of tweets with label 1.0 (sarcastic): 1909
Count of tweets with label 0.0 (non-sarcastic): 6306
